<a href="https://colab.research.google.com/github/LeeSeohyun1385/study/blob/main/%EC%98%81%ED%99%94%ED%81%AC%EB%A1%A4%EB%A7%81%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime


test_url='https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=167105&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'
resp= requests.get(test_url)
html=BeautifulSoup(resp.content, 'html.parser')

#전체 댓글수 및 페이지 수
result=html.find('div', {'class':'score_total'}).find('strong').findChildren('em')[0].getText()
comments=int(result.replace(',',''))
pages=int(np.ceil(comments/10))

#코멘트 리스트 
cmlst=pd.DataFrame(columns=['time','ID','text','score','like','dislike','watch_movie'])

#데이터 추출 함수 정의
def get_data(url):
    global cmlst
    global review_text
    global nickname
    global created_at
    global like
    global dislike
    global score
    #url 호출
    resp=requests.get(url)
    html=BeautifulSoup(resp.content,'html.parser')
    #평점 영역 추출
    score_result=html.find('div',{'class':'score_result'})
    lis=score_result.findAll('li')
    for li in lis:
        try: 
            nickname = li.findAll('a')[0].find('span').getText()
        except:
            try:
                nickname = li.findAll('a')[1].find('span').getText()
            except:
                pass
            
        try:
            created_at = datetime.strptime(li.find('dt').findAll('em')[-1].getText(), "%Y.%m.%d %H:%M")
            like=li.find('div',{'class':'btn_area'}).findAll('strong')[0].getText()
            dislike=li.find('div',{'class':'btn_area'}).findAll('strong')[1].getText()
        except AttributeError:
            pass
        
        try:
            attr = li.select_one('a')
            review_text = attr['data-src']
        except KeyError:
            review_text = li.find('p').getText().replace('\t',r'').replace('\n',r'').replace('\r',r'').replace('\t',r'').replace('관람객',r'').replace('스포일러가 포함된 감상평입니다. 감상평 보기',r'')
        score=li.find('em').getText()
        watch_movie = li.find('span', {'class':'ico_viewer'})
        cmlst=cmlst.append({'time':created_at,'ID':nickname,'text':review_text,'score':score,'like':like,
                            'dislike':dislike, 'watch_movie': watch_movie and True or False},ignore_index=True)


for i in range(1,pages+1):
    url=test_url+'&page='+str(i)
    get_data(url)

#csv출력
cmlst.to_csv('암수살인.csv',sep=',', na_rep='NaN',encoding='CP949')